In [ ]:
!pip install segmentation_models_pytorch -q

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import cv2
import matplotlib.pyplot as plt
import tifffile as tiff 
import rasterio
from rasterio.windows import Window


In [ ]:
import os
import cv2
import pdb
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
from albumentations import (HorizontalFlip, VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.pytorch import ToTensorV2
warnings.filterwarnings("ignore")


In [ ]:
# SAMPLE_SUBMISSION  = '../input/sartorius-cell-instance-segmentation/sample_submission.csv'
TRAIN_CSV = "../input/hubmap-organ-segmentation/train.csv"
# TRAIN_PATH =  "../input/hubmap-organ-segmentation/train_images/"
TRAIN_PATH =  "../temp/images/"
LABELS_PATH =  "../temp/masks/"

TEST_PATH = "../input/hubmap-organ-segmentation/test_images"
df_sample = pd.read_csv('../input/hubmap-organ-segmentation/sample_submission.csv')


RESNET_MEAN = (0.485, 0.456, 0.406)
RESNET_STD = (0.229, 0.224, 0.225)

# (336, 336)
IMAGE_RESIZE = (224, 224)

LEARNING_RATE = 5e-4
EPOCHS = 50

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

# tiled images

In [ ]:
!mkdir -p /kaggle/temp/images
!mkdir -p /kaggle/temp/masks

In [ ]:
from PIL import Image

def tile_image(p_img, folder, size: int = 1024):
    w = h = size
    im = np.array(Image.open(p_img))
    # https://stackoverflow.com/a/47581978/4521646
    tiles = [im[i:(i + h), j:(j + w), ...] for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    idxs = [(i, (i + h), j, (j + w)) for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    name, _ = os.path.splitext(os.path.basename(p_img))
    files = []
    for k, tile in enumerate(tiles):
        if tile.shape[:2] != (h, w):
            tile_ = tile
            tile = np.zeros_like(tiles[0])
            tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_
        p_img = os.path.join(folder, f"{name}_{k:02}.png")
        Image.fromarray(tile).save(p_img)
        files.append(p_img)
    return files, idxs

In [ ]:
tiles_img, _ = tile_image("../input/hubmap-organ-segmentation/train_images/12233.tiff", "/kaggle/temp/images", size=1024)
tiles_seg, idxs = tile_image("../input/hacking-the-human-body-annotation-masks/train_masks/12233.png", "/kaggle/temp/masks", size=1024)

In [ ]:
!ls -lh /kaggle/temp/images
!ls -lh /kaggle/temp/masks

In [ ]:
from skimage import color

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 9))
for i, (p_img, p_seg) in enumerate(zip(tiles_img, tiles_seg)):
    img = plt.imread(p_img)
    mask = np.array(Image.open(p_seg))
    axes[i // 3, i % 3].imshow(color.label2rgb(mask, img, bg_label=0, bg_color=(1.,1.,1.), alpha=0.25))
    axes[i // 3, i % 3].set_axis_off()
fig.tight_layout()

In [ ]:
def un_tile_image(tiles_seg, idxs, folder_image):
    tiles = [np.array(Image.open(p_seg)) for p_seg in tiles_seg]
    im = plt.imread(folder_image)
    seg = np.zeros(im.shape[:2], dtype=np.uint8)
    for tile, (i1, i2, j1, j2) in zip(tiles, idxs):
        i2 = min(i2, im.shape[0])
        j2 = min(j2, im.shape[1])
        seg[i1:i2, j1:j2] = tile[:(i2 - i1), :(j2 - j1)]
    return seg
    

In [ ]:
tiles_seg, idxs = tile_image("../input/hacking-the-human-body-annotation-masks/train_masks/12233.png", "/kaggle/temp/masks", size=1024)


In [ ]:
folder= "../input/hubmap-organ-segmentation/train_images/12233.tiff"
tiles_seg, idxs = tile_image("../input/hacking-the-human-body-annotation-masks/train_masks/12233.png", "/kaggle/temp/masks", size=1024)

reconstruted_seg = un_tile_image(tiles_seg, idxs, folder)

plt.figure(figsize=(10, 10))    
plt.imshow(reconstruted_seg)

In [ ]:
original_mask = plt.imread("../input/hacking-the-human-body-annotation-masks/train_masks/12233.png")
plt.figure(figsize=(10, 10))    
plt.imshow(original_mask)

In [ ]:
DATASET_FOLDER = "/kaggle/input/hubmap-organ-segmentation"
ANNOT_DATASET = "/kaggle/input/hacking-the-human-body-annotation-masks"

In [ ]:
[(os.path.join(DATASET_FOLDER, 'train_images'), "/kaggle/temp/images"),(os.path.join(ANNOT_DATASET, 'train_masks'), "/kaggle/temp/masks"),]

In [ ]:
from joblib import Parallel, delayed
import os, glob


TILE_SIZE = 1024

for dir_source, dir_target in [(os.path.join(DATASET_FOLDER, 'train_images'), "/kaggle/temp/images"),
    (os.path.join(ANNOT_DATASET, 'train_masks'), "/kaggle/temp/masks"),]:
    ls = glob.glob(os.path.join(dir_source, '*'))
    _= Parallel(n_jobs=3)(delayed(tile_image)(p_img, dir_target, size=TILE_SIZE) for p_img in tqdm(ls))

In [ ]:
ls = glob.glob(os.path.join(TRAIN_PATH, '*'))

In [ ]:
list_imagen = []
for image in ls:
    list_imagen.append(image[15:-4])
    
df_list = pd.DataFrame(list_imagen)
df_list.columns = ["id"]
df_list

In [ ]:
def rleToMask(rleString,height,width):
  rows,cols = height,width
  rleNumbers = [int(numstring) for numstring in rleString.split(' ')]
  rlePairs = np.array(rleNumbers).reshape(-1,2)
  img = np.zeros(rows*cols,dtype=np.uint8)
  for index,length in rlePairs:
    index -= 1
    img[index:index+length] = 255
  img = img.reshape(cols,rows)
  img = img.T
  return img

In [ ]:
class CellDatasetTiles(Dataset):
    def __init__(self, df):
        self.df = df
        self.base_path = TRAIN_PATH
        self.labels_path = LABELS_PATH

        self.transforms = Compose([Resize(IMAGE_RESIZE[0], IMAGE_RESIZE[1]), 
#                                    Normalize(mean=RESNET_MEAN, std=RESNET_STD, p=1), 
                                   HorizontalFlip(p=0.5),
                                   VerticalFlip(p=0.5),
                                   ToTensorV2()])
        self.gb = self.df.groupby('id')
        self.image_ids = df.id.unique().tolist()

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        df = self.gb.get_group(image_id)

        image_path = os.path.join(self.base_path, str(image_id) + ".png")
        image =  cv2.imread(image_path).astype('float32')
        
        label_path = os.path.join(self.labels_path, str(image_id) + ".png")
        mask =  cv2.imread(label_path)
        mask = (mask >= 1).astype('float32')
        
        augmented = self.transforms(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        return image, mask[:,:,0].reshape((1, IMAGE_RESIZE[0], IMAGE_RESIZE[1]))
#         return image, mask

    def __len__(self):
        return len(self.image_ids)

In [ ]:
class CellDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.base_path = TRAIN_PATH
        self.labels_path = LABELS_PATH

        self.transforms = Compose([Resize(IMAGE_RESIZE[0], IMAGE_RESIZE[1]), 
                                   Normalize(mean=RESNET_MEAN, std=RESNET_STD, p=1), 
                                   HorizontalFlip(p=0.5),
                                   VerticalFlip(p=0.5),
                                   ToTensorV2()])
        self.gb = self.df.groupby('id')
        self.image_ids = df.id.unique().tolist()

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        df = self.gb.get_group(image_id)
        rle = df['rle'].tolist()
        image_path = os.path.join(self.base_path, str(image_id) + ".tiff")
        image =  tiff.imread(image_path)
        mask = rleToMask(df["rle"].iloc[-1], image.shape[1], image.shape[0])
        mask = (mask >= 1).astype('float32')
        augmented = self.transforms(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        return image, mask.reshape((1, IMAGE_RESIZE[0], IMAGE_RESIZE[1]))
#         return image, mask

    def __len__(self):
        return len(self.image_ids)

In [ ]:
ds_train = CellDatasetTiles(df_list)
image, mask = ds_train[15]
image.shape, mask.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image[0].permute(1,0))
plt.imshow(mask.permute(2,1,0), alpha=0.3)
plt.show()

In [ ]:
dl_train = DataLoader(ds_train, batch_size=32, num_workers=4, pin_memory=True, shuffle=False)

In [ ]:
len(dl_train)

In [ ]:
# get a batch from the dataloader
batch = next(iter(dl_train))
images, masks = batch

In [ ]:
idx=1
plt.figure(figsize=(10,10))
plt.imshow(images[10][0].permute(1,0), cmap='bone')
plt.show()
plt.figure(figsize=(10,10))
plt.imshow(masks[10].permute(2,1,0), alpha=0.3)
plt.show()
plt.figure(figsize=(10,10))
plt.imshow(images[10][0].permute(1,0), cmap='bone')
plt.imshow(masks[10].permute(2,1,0), alpha=0.3)
plt.show()

In [ ]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))


class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()


class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

# Create model and train


In [ ]:
import torch
import collections.abc as container_abcs
torch._six.container_abcs = container_abcs
import segmentation_models_pytorch as smp

In [ ]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['cell']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device='cuda',
    verbose=True,
)

# valid_epoch = smp.utils.train.ValidEpoch(
#     model, 
#     loss=loss, 
#     metrics=metrics, 
#     device=DEVICE,
#     verbose=True,
# )

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(0, 25):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(dl_train)
#     valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
#     if max_score < valid_logs['iou_score']:
#         max_score = valid_logs['iou_score']
#         torch.save(model, './best_model.pth')
#         print('Model saved!')
    torch.save(model, './best_model.pth')

    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')

# Visualize predictions


In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [ ]:
# get a batch from the dataloader
batch = next(iter(dl_train))
images, masks = batch

In [ ]:
for i in range(30):
    
    
    image = images[i]
    gt_mask = masks[i]
#     image_vis = image.astype('uint8')
#     image, gt_mask = test_dataset[n]
    
    gt_mask = gt_mask.squeeze()
    
    x_tensor = image.to(DEVICE).unsqueeze(0)
    pr_mask = model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
        
    visualize(
        ground_truth_mask=gt_mask, 
        predicted_mask=pr_mask
    )

# Inference 

In [ ]:
df_sample = pd.read_csv('../input/hubmap-organ-segmentation/sample_submission.csv')
df_sample

In [ ]:
!mkdir -p /kaggle/temp/images_test

In [ ]:
transforms = Compose([Resize(IMAGE_RESIZE[0], IMAGE_RESIZE[1])])

un_transforms = Compose([Resize(1024, 1024)])
list_pred = []
list_idx = []

for i,row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    image_id = str(row['id'])
    im = plt.imread(os.path.join("../input/hubmap-organ-segmentation/test_images", str(image_id) + ".tiff"))
    tiles_img_pr, idxs = tile_image(os.path.join("../input/hubmap-organ-segmentation/test_images", str(image_id) + ".tiff"), 
                              "/kaggle/temp/images_test",1024)
    for p_img, idx in zip(tiles_img_pr, idxs):
        img = cv2.imread(p_img).astype('float32')
        img_trans = transforms(image=img)
        x_tensor = torch.tensor(img_trans['image']).to('cuda').permute(2,0,1)
        pr_mask = model.predict(x_tensor.unsqueeze(0))
        pr_mask = (pr_mask.squeeze().cpu().numpy().round())
        pr_mask = un_transforms(image=pr_mask)
        pr_mask = pr_mask["image"]
        list_pred.append((pr_mask))
        list_idx.append((idx))

        visualize(
            imagen=img[:,:,0], 
            predicted_mask=pr_mask
        )
        
    seg = np.zeros(im.shape[:2], dtype=np.uint8)
    for tile, (i1, i2, j1, j2) in zip(list_pred, list_idx):
        i2 = min(i2, im.shape[0])
        j2 = min(j2, im.shape[1])
        seg[i1:i2, j1:j2] = tile[:(i2 - i1), :(j2 - j1)]



In [ ]:
visualize(
            original=im,
            imagen=seg, 
        )